# Denoising by spectral substraction

The spectral subtraction method is a simple and effective method of noise reduction. In this method, an average signal spectrum and average noise spectrum are estimated in parts of the recording and subtracted from each other, so that average signal-to-noise ratio (SNR) is improved. 

The noisy signal y is a sum of the desired signal x and the noise n:
$$y = x + n$$

We proceed to a window of x, then we calculate the TF in each window: principle of the STFT (or Gabor).
We obtain :
$$Y(t,f) =X(t,f) +N(t,f),∀(t,f)$$
x and n being independent, we obtain :
$$E[|Y(t,f)|^2]=E[|X(t,f)|^2]+S_n(f)$$
Which gives as estimator : 
$$E[|X(t,f)|^2]=E[|Y(t,f)|^2]−S_n(f)$$
##### digonal estimator 
$$\hat{X}(t,f) =A(t,f)Y(t,f)$$
Where :
$$A(t,f) = (\dfrac{|Y(t,f)|^{\alpha} - \lambda S_n(f)}{|Y(t,f)|^{\alpha}})^{\beta}$$
With : 
* $\alpha=2$ and $\beta = 0.5$ : power spectral substraction
* $\alpha=1$ and $\beta = 1$ : magnitude spectral substraction

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from scipy.signal import lfilter, stft, istft
from pydub import AudioSegment


eps = sys.float_info.epsilon

In [2]:
# the maximal frequency Fs is equal for the three noise signal
Fs_n, noise_1 = wav.read('noise1.wav')
Fs_n, noise_2 = wav.read('noise2.wav')
Fs_n, noise_3 = wav.read('noise3.wav')

Fs_m, music = wav.read('music.wav')

# noise signal
k = 0.5
noise = k *(noise_1 + noise_2 + noise_3)

# noisy signal 
noised_music = music +  noise

# Input SNR
val= np.var(noised_music)/np.var(noise)
Input_snr = 10 * np.log10(val)

print(Input_snr)

2.8241005511357398


In [3]:
# Parameters
alpha = 2
beta = 0.5
Lambda = 1.

In [4]:
# short-time Fourier transform of the noisy signal
freq, tps, noised_music_stft = stft(noised_music, Fs_m, nperseg=1024)
# short-time Fourier transform of the original signal
_,_,music_stft = stft(music, Fs_m, nperseg=1024)

In [5]:
# Noise estimate

freq, tps, noise_stft = stft(noise, Fs_n, nperseg=1024)
#  we calculate the temporal mean for every frame
snf = np.mean(np.power(noise_stft,alpha),axis=1)
snf = np.transpose(np.repeat([snf], len(noise_stft[0]), axis=0))


In [6]:
numer_A = np.power(noised_music_stft,alpha)-Lambda * snf
denom_A = np.power(noised_music_stft, alpha)+eps
A = np.power(numer_A/denom_A, beta)
denoised_music_stft = A * noised_music_stft
tps, denoised_music = istft(denoised_music_stft, Fs_m, nperseg=1024)

In [7]:
wav.write("denoised_music.wav", Fs_n, np.int16(denoised_music))
wav.write("noised_music.wav", Fs_n, np.int16(noised_music))

In [8]:
Original_music = AudioSegment.from_wav('music.wav')
Noisy_music = AudioSegment.from_wav("noised_music.wav")
Denoised_music = AudioSegment.from_wav("denoised_music.wav")

In [9]:
print('Original music :')
Original_music

Original music :


In [10]:
print('Noisy music :')
Noisy_music

Noisy music :


In [11]:
print('Denoised music :')
Denoised_music

Denoised music :


In [12]:
# Output SNR
coef= np.var(denoised_music)/np.var((music-denoised_music[:len(music)]))
output_snr = 10 * np.log10(coef)
print(output_snr)

2.883361147431037


####  Remarks :

* The Signal-to-noise ratio (SNR) is ameliorated after applying the spectral substraction to the noisy signal.
* The power spectral substraction give a better result than the magnitude spectral substraction.